In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Dataset ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path)
        self.y = np.argmax(np.load(y_path), axis=1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y

# ---------------- CNN Encoder ----------------
class CNNEncoder(nn.Module):
    def __init__(self, emb_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(256, emb_dim)  # map CNN output to embedding

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, channels, seq_len)
        x = self.net(x).squeeze(-1)  # (B, 256)
        z = F.normalize(self.fc(x), dim=1)
        return z

# ---------------- Full Model ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNNEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- Margin Embedding Loss ----------------
class MarginEmbeddingLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, z, y):
        dist = torch.cdist(z, z, p=2)
        y = y.unsqueeze(1)
        pos_mask = (y == y.T).float()
        neg_mask = (y != y.T).float()
        eye = torch.eye(len(z), device=z.device)
        pos_mask = pos_mask - eye
        pos_loss = pos_mask * dist.pow(2)
        neg_loss = neg_mask * F.relu(self.margin - dist).pow(2)
        loss = (pos_loss.sum() + neg_loss.sum()) / (pos_mask.sum() + neg_mask.sum() + 1e-8)
        return loss

# ---------------- Early Stopping ----------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- Training & Evaluation ----------------
def train_and_eval(model, train_loader, test_loader, margin_loss, alpha=1.0, epochs=50, patience=10):
    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            z, logits = model(x)
            loss = ce_loss(logits, y) + alpha * margin_loss(z, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(dim=1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:02d} | Test Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping triggered")
            break

    stopper.restore(model)
    return stopper.best_acc

# ---------------- Main Loop ----------------
BASE = "models/Data/Data/62_classes/UserIndependent"
margin_loss = MarginEmbeddingLoss(margin=1.0)
all_accs = []

for split in range(1, 20):
    print(f"\n========== SPLIT {split} ==========")
    train_ds = EMGDataset(f"{BASE}/Train/X_train_{split}.npy", f"{BASE}/Train/y_train_{split}.npy")
    test_ds = EMGDataset(f"{BASE}/Test/X_test_{split}.npy", f"{BASE}/Test/y_test_{split}.npy")

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

    model = FullModel(emb_dim=128).to(device)

    acc = train_and_eval(model, train_loader, test_loader, margin_loss, alpha=1.0, epochs=50, patience=10)
    print(f"Best Split Accuracy: {acc:.4f}")
    all_accs.append(acc)

    del model
    torch.cuda.empty_cache()

print("FINAL RESULT ")
print("Average Accuracy (Margin + CE):", np.mean(all_accs))


Using device: cuda

========== SPLIT 1 ==========
Epoch 01 | Test Acc: 0.0516
Epoch 02 | Test Acc: 0.1806
Epoch 03 | Test Acc: 0.3290
Epoch 04 | Test Acc: 0.4565
Epoch 05 | Test Acc: 0.5113
Epoch 06 | Test Acc: 0.5210
Epoch 07 | Test Acc: 0.6032
Epoch 08 | Test Acc: 0.6113
Epoch 09 | Test Acc: 0.5887
Epoch 10 | Test Acc: 0.6581
Epoch 11 | Test Acc: 0.6371
Epoch 12 | Test Acc: 0.6145
Epoch 13 | Test Acc: 0.6419
Epoch 14 | Test Acc: 0.6387
Epoch 15 | Test Acc: 0.6419
Epoch 16 | Test Acc: 0.6258
Epoch 17 | Test Acc: 0.6210
Epoch 18 | Test Acc: 0.6403
Epoch 19 | Test Acc: 0.6081
Epoch 20 | Test Acc: 0.6306
Early stopping triggered
Best Split Accuracy: 0.6581

========== SPLIT 2 ==========
Epoch 01 | Test Acc: 0.0306
Epoch 02 | Test Acc: 0.0952
Epoch 03 | Test Acc: 0.1726
Epoch 04 | Test Acc: 0.3113
Epoch 05 | Test Acc: 0.3839
Epoch 06 | Test Acc: 0.4355
Epoch 07 | Test Acc: 0.4484
Epoch 08 | Test Acc: 0.3968
Epoch 09 | Test Acc: 0.4371
Epoch 10 | Test Acc: 0.4468
Epoch 11 | Test Acc: 0.422

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ---------------- Setup ----------------
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Dataset ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.argmax(np.load(y_path), axis=1)

        # channel-wise normalization (VERY IMPORTANT)
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM Model ----------------
class CNN_BiLSTM(nn.Module):
    def __init__(self, num_classes=62, emb_dim=256):
        super().__init__()

        # CNN
        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        # BiLSTM
        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        # Embedding + Classifier
        self.embedding = nn.Linear(256, emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)          # (B, C, T)
        x = self.cnn(x)                 # (B, 256, T')
        x = x.permute(0, 2, 1)          # (B, T', 256)

        lstm_out, _ = self.bilstm(x)
        feat = lstm_out[:, -1, :]       # last timestep

        emb = self.embedding(feat)
        logits = self.classifier(emb)

        return emb, logits
    
class EarlyStopping:
    def __init__(self, patience=15, mode="max"):
        self.patience = patience
        self.mode = mode
        self.best = None
        self.counter = 0
        self.best_state = None

    def step(self, value, model):
        improve = (
            value > self.best if self.mode == "max"
            else value < self.best
        ) if self.best is not None else True

        if improve:
            self.best = value
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

def train_stage1(model, train_loader, test_loader, epochs=50):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)

    best_acc = 0
    best_state = None
    patience, counter = 8, 0

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            _, logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # validation
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"[Stage-1] Epoch {epoch+1:02d} | Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break

    model.load_state_dict(best_state)

class MarginLoss(nn.Module):
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin

    def forward(self, z, y):
        z = F.normalize(z, dim=1)
        dist = torch.cdist(z, z, p=2)

        y = y.unsqueeze(1)
        pos = (y == y.T).float()
        neg = (y != y.T).float()

        eye = torch.eye(len(z), device=z.device)
        pos = pos - eye

        pos_loss = pos * dist.pow(2)
        neg_loss = neg * F.relu(self.margin - dist).pow(2)

        return (pos_loss.sum() + neg_loss.sum()) / (pos.sum() + neg.sum() + 1e-8)

def train_stage2(model, train_loader, test_loader, margin_loss, epochs=20):
    # 🔒 freeze classifier
    for p in model.classifier.parameters():
        p.requires_grad = False

    optimizer = Adam(model.encoder.parameters(), lr=1e-4)

    best_acc = 0
    best_state = None
    patience, counter = 6, 0

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            z, _ = model(x)
            loss = margin_loss(z, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct = total = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"[Stage-2] Epoch {epoch+1:02d} | Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break

    model.load_state_dict(best_state)

margin_loss = MarginLoss(margin=1.0)
all_accs = []

for split in range(1, 20):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(f"{BASE}/Train/X_train_{split}.npy",
                          f"{BASE}/Train/y_train_{split}.npy")
    test_ds = EMGDataset(f"{BASE}/Test/X_test_{split}.npy",
                         f"{BASE}/Test/y_test_{split}.npy")

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = FullModel(emb_dim=128).to(device)

    train_stage1(model, train_loader, test_loader)
    train_stage2(model, train_loader, test_loader, margin_loss)

    # final eval
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            _, logits = model(x)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    acc = correct / total
    print(f"Final Acc (Split {split}): {acc:.4f}")
    all_accs.append(acc)

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVG ACC:", np.mean(all_accs))

BASE = "models/Data/Data/62_classes/UserIndependent"
final_accs = []

for split in range(1, 20):
    print(f"\n========== SPLIT {split} ==========")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = CNN_BiLSTM(num_classes=62).to(device)

    # Stage-1
    model, best_acc = train_stage1(
        model,
        train_loader,
        test_loader,
        epochs=500,
        patience=10
    )

    print(f"Best Stage-1 Accuracy: {best_acc:.4f}")

    # Stage-2
    model = train_stage2(
        model,
        train_loader,
        epochs=50,
        patience=10
    )

    final_accs.append(best_acc)

    del model
    torch.cuda.empty_cache()

print("\nFINAL AVERAGE ACCURACY:", np.mean(final_accs))


Using device: cuda

===== SPLIT 1 =====
[Stage-1] Epoch 01 | Acc: 0.0500
[Stage-1] Epoch 02 | Acc: 0.0758
[Stage-1] Epoch 03 | Acc: 0.0968
[Stage-1] Epoch 04 | Acc: 0.1290
[Stage-1] Epoch 05 | Acc: 0.1403
[Stage-1] Epoch 06 | Acc: 0.1548
[Stage-1] Epoch 07 | Acc: 0.2065
[Stage-1] Epoch 08 | Acc: 0.2210
[Stage-1] Epoch 09 | Acc: 0.2435
[Stage-1] Epoch 10 | Acc: 0.3016
[Stage-1] Epoch 11 | Acc: 0.3274
[Stage-1] Epoch 12 | Acc: 0.3613
[Stage-1] Epoch 13 | Acc: 0.3968
[Stage-1] Epoch 14 | Acc: 0.4177
[Stage-1] Epoch 15 | Acc: 0.4306
[Stage-1] Epoch 16 | Acc: 0.4371
[Stage-1] Epoch 17 | Acc: 0.4065
[Stage-1] Epoch 18 | Acc: 0.4758
[Stage-1] Epoch 19 | Acc: 0.5032
[Stage-1] Epoch 20 | Acc: 0.5323
[Stage-1] Epoch 21 | Acc: 0.4984
[Stage-1] Epoch 22 | Acc: 0.5274
[Stage-1] Epoch 23 | Acc: 0.5500
[Stage-1] Epoch 24 | Acc: 0.5387
[Stage-1] Epoch 25 | Acc: 0.5452
[Stage-1] Epoch 26 | Acc: 0.5177
[Stage-1] Epoch 27 | Acc: 0.5516
[Stage-1] Epoch 28 | Acc: 0.5839
[Stage-1] Epoch 29 | Acc: 0.5613
[St

TypeError: train_stage1() got an unexpected keyword argument 'patience'

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ---------------- Setup ----------------
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Dataset ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.argmax(np.load(y_path), axis=1)

        # channel-wise normalization
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM Encoder ----------------
class CNNBiLSTMEncoder(nn.Module):
    def __init__(self, emb_dim=128):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)        # (B, C, T)
        x = self.cnn(x)               # (B, 256, T')
        x = x.permute(0, 2, 1)        # (B, T', 256)

        lstm_out, _ = self.bilstm(x)
        feat = lstm_out[:, -1, :]     # last timestep

        z = F.normalize(self.fc(feat), dim=1)
        return z

# ---------------- Full Model ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=128, num_classes=62):
        super().__init__()
        self.encoder = CNNBiLSTMEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- Margin Embedding Loss ----------------
class MarginEmbeddingLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, z, y):
        dist = torch.cdist(z, z, p=2)

        y = y.unsqueeze(1)
        pos_mask = (y == y.T).float()
        neg_mask = (y != y.T).float()

        eye = torch.eye(len(z), device=z.device)
        pos_mask = pos_mask - eye

        pos_loss = pos_mask * dist.pow(2)
        neg_loss = neg_mask * F.relu(self.margin - dist).pow(2)

        loss = (pos_loss.sum() + neg_loss.sum()) / (pos_mask.sum() + neg_mask.sum() + 1e-8)
        return loss

# ---------------- Early Stopping ----------------
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- Training & Evaluation ----------------
def train_and_eval(model, train_loader, test_loader, margin_loss,
                   alpha=1.0, epochs=500, patience=10):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)
            loss = ce_loss(logits, y) + alpha * margin_loss(z, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:02d} | Test Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping triggered")
            break

    stopper.restore(model)
    return stopper.best_acc

# ---------------- Main Loop ----------------
BASE = "models/Data/Data/62_classes/UserIndependent"
margin_loss = MarginEmbeddingLoss(margin=1.0)
all_accs = []

for split in range(1, 20):
    print(f"\n========== SPLIT {split} ==========")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

    model = FullModel(emb_dim=128, num_classes=62).to(device)

    acc = train_and_eval(
        model,
        train_loader,
        test_loader,
        margin_loss,
        alpha=1.0,
        epochs=500,
        patience=10
    )

    print(f"Best Split Accuracy: {acc:.4f}")
    all_accs.append(acc)

    del model
    torch.cuda.empty_cache()

print("\nFINAL RESULT")
print("Average Accuracy (CNN+BiLSTM + CE + Margin):", np.mean(all_accs))


Using device: cuda

========== SPLIT 1 ==========
Epoch 01 | Test Acc: 0.0548
Epoch 02 | Test Acc: 0.0871
Epoch 03 | Test Acc: 0.0887
Epoch 04 | Test Acc: 0.1516
Epoch 05 | Test Acc: 0.1935
Epoch 06 | Test Acc: 0.2629
Epoch 07 | Test Acc: 0.2984
Epoch 08 | Test Acc: 0.3129
Epoch 09 | Test Acc: 0.3419
Epoch 10 | Test Acc: 0.3661
Epoch 11 | Test Acc: 0.3806
Epoch 12 | Test Acc: 0.4258
Epoch 13 | Test Acc: 0.4484
Epoch 14 | Test Acc: 0.4774
Epoch 15 | Test Acc: 0.4903
Epoch 16 | Test Acc: 0.3629
Epoch 17 | Test Acc: 0.4613
Epoch 18 | Test Acc: 0.5226
Epoch 19 | Test Acc: 0.5113
Epoch 20 | Test Acc: 0.5435
Epoch 21 | Test Acc: 0.5032
Epoch 22 | Test Acc: 0.5645
Epoch 23 | Test Acc: 0.5887
Epoch 24 | Test Acc: 0.5694
Epoch 25 | Test Acc: 0.6032
Epoch 26 | Test Acc: 0.5677
Epoch 27 | Test Acc: 0.5468
Epoch 28 | Test Acc: 0.5758
Epoch 29 | Test Acc: 0.5710
Epoch 30 | Test Acc: 0.6000
Epoch 31 | Test Acc: 0.6081
Epoch 32 | Test Acc: 0.5903
Epoch 33 | Test Acc: 0.5806
Epoch 34 | Test Acc: 0.601